# Lab 6a: Decision Tree Classifiers

In this lab, you will be analyzing the implementation of a simple decision tree classifier. The goal of Lab 6a is to help you understand  decision tree design.  This understanding will help you effectively optimize decision tree hyperparameters when modeling real data.

---
## Review
### What is a Decision Tree?
A *Decision Tree* is  tree-shaped graph or model of decisions used to determine a course of action or show a statistical probability.  

In context of supervised machine learning two types of decision tree are commonly employed:

- **Classification Tree:** A decision tree that performs classification (predicts a categorical response).
- **Regression Tree:** A decision tree that performs regression (predicts a numeric response).

Though it is possible to have have hybrid decision trees that provide both types of responses.

Below is a classification tree for classifying patients as either "Low risk" or "High risk" for a heart attack based on their medical information.

<img src="https://onlinecourses.science.psu.edu/stat857/sites/onlinecourses.science.psu.edu.stat857/files/lesson13/image_01.gif">

Classification works as follows, starting at the top of diagram (i.e., the root of the tree), and check whether or not the medical information present satisfies the _rule_ associated with each circle (i.e., _internal node_ or simply _node_) until we reach a square (i.e., _terminal node_, _leaf node_, or simply _leaf_). The patients class ("Low risk", or "High risk") is then provided by the _label_ associated with that square.

When considered in the context of a set of data (e.g., training data), each _internal node_'s decision rule splits the set of data into two subsets, therefore _internal node_'s are also sometimes referred to as _split points_.

An beautiful presentation on how ML Decision Trees work, can be found here [A Visual Introduction to Machine Learning](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/)

### Task 1
For the classification tree below, answer the following questions:
1. How many _internal nodes_ and _leaf nodes_ are need to represent the tree below.
2. What decision _rules_ are present?
3. Is all the feature data ordinal or is some it categorical?
4. What are the possible _labels_ the tree can produce?
<img src="images/credit_tree.png" width=500>

A pythonic tree class is implemented below. Skim the code below to get a general understanding of how you can code a decision tree in Python.

In [ ]:
class Data1030Tree:
    """An empty base class. Used to implement attributes shared by
    Data1030Node and Data1030Leaf.
    """
    pass

class Data1030Leaf(Data1030Tree):
    """A leaf in a decision tree

    Parameters:
    -----------
    label: int
       The label of the leaf.
    """
    def __init__(self, label: int):
        self.label = label

class Data1030Node(Data1030Tree):
    """A node in a decision tree

    Parameters:
    -----------
    feature: int
        The integer index of the feature used to split the node.

    threshold: float
        The value to split the feature by. If the threshold is 2.5,
        then all samples with values < 2.5 will be in the lower split
        while all samples with values <= 2.5 will be in the upper split.

    left_child: Data1030Tree
        The subtree formed by samples below the threshold.

    right_child: Data1030Tree
        The subtree formed by samples above the threshold.
    """
    def __init__(self,
                 feature: int,
                 threshold: float,
                 left_child: Data1030Tree,
                 right_child: Data1030Tree):
        self.feature = feature
        self.threshold = threshold
        self.left_child = left_child
        self.right_child = right_child

In [ ]:
def tree_to_string(dtree: Data1030Tree, depth=0):
    """Prints the tree in a human readable form.
    """
    if isinstance(dtree, Data1030Leaf):
        tree_str = '\t' * depth
        tree_str += 'Leaf: [Label] = {}\n'.format(dtree.label)
        return tree_str
    
    if isinstance(dtree, Data1030Node):
        tree_str = ''

        tree_str += '\t' * depth
        tree_str += 'Node: [Feature {}] < {}\n'.format(dtree.feature, dtree.threshold)
        tree_str += tree_to_string(dtree.left_child, depth=depth + 1)

        tree_str += '\t' * depth
        tree_str += 'Node: [Feature {}] >= {}\n'.format(dtree.feature, dtree.threshold)
        tree_str += tree_to_string(dtree.right_child, depth=depth + 1)
        return tree_str
        
# Sample points x, with x[3] <= 10 are categorized to label 2 by example_tree
# Sample points x, with x[3] > 10 are categorized to label 1 by example_tree
example_tree = Data1030Node(1,-3, Data1030Node(3, 10, Data1030Leaf(2), Data1030Leaf(1)), Data1030Leaf(3))
print(tree_to_string(example_tree))

You do not need to completely understand the details of the above implementation but do note
that leafs and nodes are distinguished. As we traverse a decision tree, we are either
"splitting" based on a rule (like Age < 62.5) or giving the data a label (like High Risk).

---
## How do we construct the "right" tree?

Given a tree, we know how to classify an observation. So how do we create the right tree?

Creating a decision tree requires selecting input variables and split points on those variables until a suitable tree is constructed.

There are many algorithms for doing this, each with tradeoffs when it comes to memory, generalizability, and performance.

Breiman's original Classification and Regression Trees (CART) algorithm provides a good starting point. CART is a "greedy algorithm" that recursively splits the feature space rectilinearly until most of the data in a given hyperrectangle (box) has a certain label.  

The algorithm is greedy in that the decision rule (feature and cut-off criteria) selected at each split point is the one that best minimizes a cost function applied to the current data. There is no back-tracking or look-ahead.

If you run CART on a 2-D classification problem (i.e. one with two continuous features), and plot the decision boundaries for the resulting classification tree, you will get something like the image below.

<img src="https://docs.microsoft.com/en-us/azure/machine-learning/studio/media/algorithm-choice/image5.png"
     width=300>
     
In the classification tree above, note how each _leaf_ in the decision tree below corresponds to a different _box_ in the decision boundary plot above it.

### Task 2

Suppose that we swapped the positions of the node __Income < 61k__ and the node __Age < 27__ in the tree above. 

1. Would any data points in the above graph be classified differently? 2. Describe what would change about green lines in the coordinate graph above.

### Breiman's CART Algorithm
To _greedily_ create the tree, CART does the following:

1. (Stopping Condition) Check if the input data, X, are mostly of a certain label, if so create a leaf with that label.
2. (Split-node Selection) If not, split the data based on the rule (ex. Age < 27) that best minimizes splitting costs. 
3. Run steps 1 and 2 recursively on each split.

_Gini impurity_, $G(y)$, is defined as as follows:

Given a labeled data set (X,y) with $k$ possible class labels. Let $p_i$ be the proportion of observations (y) with class label $i$.

$$G(y) = \sum_{i = 1}^{k}p_i (1 - p_i) = 1 - \sum_{i = 1}^{k}p_i^2$$

A _weighted sum_ of Gina impurity measures is used to evaluate the cost of each possible split.  Given a split of (X,y) into $(X^1,y^1)$ and $(X^2,y^2)$, the $cost$ of this split is 
$$cost(y^1, y^2) = \frac{|y^1|}{|y|}g(y^1)| + \frac{|y^2|}{|y|}g(y^2))$$


### Example Implementation
The Data1030TreeBuilder code below contains a `.build` method 
that implements CART that allows the user to select 
either 'gini' or 'entropy' for decision criteria evaluation.

In [ ]:
import numpy as np

class Data1030TreeBuilder:
    """Constructs a decision tree (a collection of connected
    Data1030Node and Data1030Leaf objects)

    To construct a decision tree, instantiate a
    Data1030TreeBuilder object with no parameters
    and then call the .build method with the training
    data and training labels as arguments.
    """

    def build(self,
              X: np.array,
              y: np.array,
              criterion: str) -> Data1030Tree:
        """Returns a Data1030 tree from the input data."""
        # Checks whether or not all of the sample labels are the same
        if np.all(y == y[0]):
            return Data1030Leaf(y[0])

        # Finds the pair (attribute, split_value) with the highest
        # information gain
        attribute, threshold = self.best_split(X, y, criterion)

        # Calls self.build on row pairs in X, y which are
        # below the threshold
        left_child = self.build(X[X[:, attribute] < threshold],
                                y[X[:, attribute] < threshold],
                                criterion)

        # Calls self.build on row pairs in X, y which are
        # above or equal to the threshold
        right_child = self.build(X[X[:, attribute] >= threshold],
                                 y[X[:, attribute] >= threshold],
                                 criterion)

        return Data1030Node(attribute, threshold, left_child, right_child)

    def best_split(self,
                   X: np.array,
                   y: np.array,
                   criterion='gini') -> (int, float):
        """Returns the attribute and threshold that result in the
        optimal split. This occurs when the gini impurity or
        entropy of the resulting subsets is minimized.
        """
        best_attribute = -1
        best_threshold = - 1
        best_impurity_loss = 1
        for attribute in range(X.shape[1]):
            # The lowest unique element cannot be a threshold value
            # as the split must split X into 2 non-empty collections
            possible_thresholds = np.unique(X[:, attribute])[1:]
            for threshold in possible_thresholds:
                # Chooses the criterion to use
                if criterion is 'gini':
                    current_impurity_loss = self.gini(X,
                                                      y,
                                                      attribute,
                                                      threshold)
                elif criterion is 'entropy':
                    current_impurity_loss = self.entropy(X,
                                                         y,
                                                         attribute,
                                                         threshold)
                else:
                    raise ValueError(
                        '{} is not a valid criterion'.format(criterion))
                # Checks to see whether the current split results in lower loss
                if current_impurity_loss < best_impurity_loss:
                    best_attribute = attribute
                    best_threshold = threshold
                    best_impurity_loss = current_impurity_loss

        return best_attribute, best_threshold

    def gini(self,
             X: np.array,
             y: np.array,
             attribute: int,
             threshold: float) -> float:
        """Calculates the weighted sum of the gini impurities of the candidate
        left and right children.
        """
        # Let arr be an Numpy array of integers
        # Calling arr[arr < 5] returns all elements in less than 5
        # In the above scenario, arr < 5 is a "mask"
        masks = (X[:, attribute] < threshold, X[:, attribute] >= threshold)
        gini_sum = 0
        for mask in masks:
            # The line below counts the number of elements in each
            # output class/label
            counts = np.unique(y[mask], return_counts=True)[1]
            proportions = counts / len(y[mask])
            gini_impurity = 1 - np.dot(proportions, proportions)
            # The weight is the proportion of elements in the subset, y[mask],
            gini_weight = len(y[mask]) / len(y)
            gini_sum += gini_weight * gini_impurity
        return gini_sum

    def entropy(self,
                X: np.array,
                y: np.array,
                attribute: int,
                threshold: float) -> float:
        """Calculates the weighted sum of the entropies of the candidate
        left and right children.
        
        If you wish to consult information theory resources,
        this is the conditional entropy:
        
            H(X|RULE) = p(RULE=True) * H(X|Rule=True) + p(RULE=False) * H(X|Rule=False)
        """
        # TASK 3 HERE
        # BEGIN SOLUTION
        masks = (X[:, attribute] < threshold, X[:, attribute] >= threshold)
        entropy_sum = 0
        for mask in masks:
            counts = np.unique(y[mask], return_counts=True)[1]
            proportions = counts / len(y[mask])
            conditional_entropy = -np.dot(proportions, np.log2(proportions))
            entropy_weight = len(y[mask]) / len(y)
            entropy_sum += entropy_weight * conditional_entropy
        return entropy_sum
        # END SOLUTION

### Task 3
Explain how best_split operates.  How is the returned 
(attribute index, threshold) found? Include a description of the number of threshold values that are evaluated.

###  Entropy Measure
_Entropy_, $E(y)$ is another common measure used for split-point selection.  It is defined as follows:

$$E(y) = \sum_{i = 0}^{k} p_i \log \frac{1}{p_i}$$

where, as before, $p_i$ is the proportion of observations (y) with class label $i$.

The more "variable" a random variable is, the higher its value of _gini impurity_ and _entropy_ will be. 

### Task 4
Complete the `.entropy()` method in the `Data1030TreeBuilder` code above so that returns the cost of a split using Entropy. Use the implementation of Gini impurity to help you. Below is some test code to see if you have it right.

In [ ]:
def test_entropy():
    builder = Data1030TreeBuilder()
    y = np.random.randint(2, size=10)*10
    print('y = ', y)
    X = np.column_stack((np.random.randint(2, size=10)*10, y)) # Second column of X contains y
    print('X = ', X)
    cost_split_1 = builder.entropy(X, y, 0, 5) # Try splitting using X[:,0] < 5
    cost_split_2 = builder.entropy(X, y, 1, 5) # Try splitting using X[:,1] < 5
    print('cost_split_1 =', cost_split_1)
    print('cost_split_2 =', cost_split_2)
    assert cost_split_1 >= 0 # cost of random split  
    assert cost_split_2 == 0 # cost of a perfect split is zero

    
test_entropy()


Below is an example of a fully functional TreeClassfier, with .fit() and .predict() methods.  Notice how clean the design is.  It may seem surprising how powerful they are, given how straightforward they are to work with.  Soon, you will learn about Deep Learning models that have even simpler implementations.

In [ ]:
class Data1030TreeClassifier(Data1030TreeBuilder):
    """A simple decision tree classifier based on sklearn's design

    To use the class, instantiate a Data1030TreeClassifier object
    and then call the .fit method to train the model. 

    Attributes:
    -----------
    tree_:
        The underlying DATA1030Tree used to get predicted labels. Set to 
        none if the tree is not trained.
    """

    def __init__(self):
        self.tree_ = None

    def fit(self, X: np.array, y: np.array, criterion='gini') -> None:
        """Trains the model on the training data X and
        training labels y. 

        Categorical features should be encoded as integers.
        The output labels should be encoded as integers.
        """
        self.tree_ = self.build(X, y, criterion)

    def predict(self, X: np.array) -> np.array:
        """Predicts the labels y-hat of of the given inputs X.
        
        The input features X should be encoded in the same
        format as the training features.
        """
        if self.tree_ is None:
            msg = ('This decision tree has not been fitted. Call "fit" with '
                   'appropriate arguments before using this method.')
            raise ValueError(msg)

        predictions = np.zeros(shape=len(y_train), dtype=np.int)
        for i in range(len(X)):
            cursor = self.tree_
            # Traverses the tree according to the rule in the node
            # until cursor reaches a leaf
            while isinstance(cursor, Data1030Node):
                if X[i, cursor.feature] < cursor.threshold:
                    cursor = cursor.left_child
                else:
                    cursor = cursor.right_child
            predictions[i] = cursor.label
        return predictions

CART can also be used to do regression. [This guide](https://sadanand-singh.github.io/posts/treebasedmodels/) is a good introduction to CART for regression.

---
## Applying _sklearn_ decision trees

In practice, you will spend a lot more time prototyping ML models with _sklearn_ than coding models from scratch. However _sklearn's_ implementation of CART is a bit hard to handle. For example, `sklearn.tree.DecisionTreeClassifier` has 13 parameters!

If you tune those parameters improperly, you likely will end up with an _overfitted_ model like the one graphed below.

<img src="http://stephanie-w.github.io/brainscribble/figure/classification-algorithms-on-iris-dataset_48_0.png">

Before we dive into the parameters, install the `python-graphviz` conda package by typing the following in your terminal:

`conda install python-graphviz`

Once you have installed `python-graphviz`, _restart your kernel_ and run the code block to create a diagram of an iris classification tree.

**Take some time to understand what each row in each node represents.**

In [ ]:
import graphviz
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# Loads your feature data to X and label data to y
iris = load_iris()
X = iris.data
y = iris.target

# Fits your decision tree to the iris data
clf = DecisionTreeClassifier()
clf.fit(X, y)

# Graphs your tree
dot_data = export_graphviz(clf, 
                           feature_names = iris.feature_names, # optional 
                           class_names  = iris.target_names, # optional
                           out_file=None) 
graph = graphviz.Source(dot_data) 
graph 

Here is a list of important `DecisionTreeClassifier` parameters from [dataaspirant.com](dataaspirant.com). It is a good idea to be familiar with many of these.

* __criterion:__ It defines the function to measure the quality of a split. Sklearn supports “gini” criteria for Gini Index & “entropy” for Information Gain. By default, it takes “gini” value.
* __splitter:__ It defines the strategy to choose the split at each node. Supports “best” value to choose the best split & “random” to choose the best random split. By default, it takes “best” value.
* __max_features:__ It defines the no. of features to consider when looking for the best split. We can input integer, float, string & None value.
    If an integer is inputted then it considers that value as max features at each split.
    If float value is taken then it shows the percentage of features at each split.
    If “auto” or “sqrt” is taken then max_features=sqrt(n_features).
    If “log2” is taken then max_features= log2(n_features).
    If None, then max_features=n_features. By default, it takes “None” value.
* __max_depth:__ The max_depth parameter denotes maximum depth of the tree. It can take any integer value or None. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples. By default, it takes “None” value.
* __min_samples_split:__ This tells above the minimum no. of samples reqd. to split an internal node. If an integer value is taken then consider min_samples_split as the minimum no. If float, then it shows percentage. By default, it takes “2” value.
* __min_samples_leaf:__ The minimum number of samples required to be at a leaf node. If an integer value is taken then consider min_samples_leaf as the minimum no. If float, then it shows percentage. By default, it takes “1” value.
* __max_leaf_nodes:__ It defines the maximum number of possible leaf nodes. If None then it takes an unlimited number of leaf nodes. By default, it takes “None” value.
* __min_impurity_decrease:__ It defines the threshold for early stopping tree growth. A node will split if its impurity is above the threshold otherwise it is a leaf.

### Task 4

Categorize the above parameters into (1) those that directly affect _when_ the tree stops splitting (2) those that directly affect _how_ the tree splits.

### Task 5
If you want to reduce the overfitting of your model would you increase or decrease the following parameters?

* max_depth
* min_samples_split
* min_samples_leaf
* max_leaf_nodes
* min_impurity_decrease

### Task 6
Run the code block below and compare the classification tree here with the one above.  

Answer the following:
1. Is the training accuracy here below _better_ or _worse_ than the one above?  
2. What is the training accuracy for the try below?  What is it for the tree above?

Next, try adjusting the parameters of DecisionTreeClassifier instance assigned to `clf2` to see how your tree responds to changes to each parameter.  The parameters are pretty self-explanatory, but feel free to check the sklearn [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) documentation as well.

Note: if you get an error about `min_impurity_decrease` upgrade your sklearn-learn via
```
conda install scikit-learn
```

In [ ]:
clf2 = DecisionTreeClassifier(max_depth=2,
                              min_samples_split=2,
                              max_leaf_nodes=10,
                              min_impurity_decrease=0)
clf2.fit(X, y)

# Graphs your tree
dot_data2 = export_graphviz(clf2, 
                            feature_names = iris.feature_names, # optional 
                            class_names  = iris.target_names, # optional
                            out_file=None) 
graph2 = graphviz.Source(dot_data2) 
graph2 

## Task 7

True or False:

1. Overfitting isn't a problem with decision trees.
2. It is always possible to perfectly overfit the data using Decision trees.
3. Do you need to normalize numerical features before calling DecisionTreeClassifier.fit?
4. Because CART greedily chooses the best split, will a decision tree built by CART always have a minimum total cost? (i.e. will they be optimal in that sense?)

If you are stuck read the _sklearn_ [User Guide](http://scikit-learn.org/stable/modules/tree.html) on decision trees.

## Some Take Home Messages
* Decision trees are typically binary trees. Each node has a rule (like age < 10) and each leaf has a label (e.g. went to burning man = True).
* _sklearn_ decision trees have many optional parameters. How one adjusts them can help deal with overfitting.
* _sklearn_ decision trees do not require you to preprocess your data much and can be visualized with `graphviz`.